In [1]:
import numpy as np

In [2]:
def linear_congurence_next(a,x,b,m):
    return (a*x+b)%m

In [3]:
mu=31
nu=13
m=16384
b=1
a=5
x=3
vals=[x/m]
xs=[x]
for i in range(mu):
    x=linear_congurence_next(a,x,b,m)
    vals.append(x/m)
    xs.append(x)
    
for i in range(100_000):
    xs.append((xs[-mu]-xs[-nu])%m)
    vals.append(xs[-1]/m)
    
    

In [4]:
def poisson_generator(lamda,u):
    x=0
    p=np.exp(-lamda)
    s=p
    while u>s:
        x+=1
        p*=lamda/x
        s+=p
    return x

In [5]:
def weibull_generator(k,scale,u):
    return (-np.log(u+1e-9)/scale)**(1/k)
    

In [6]:
lamda=2.9

In [7]:
poi_vals1=[]
poi_vals2=[]
for i in range(100):
    poi_vals1.append(poisson_generator(lamda,vals[i]))
for i in range(i,i+10000):
    poi_vals2.append(poisson_generator(lamda,vals[i]))





In [8]:
k=0.8
scale=3**(-0.8)

### Simple Monte Carlo

In [9]:
t=i
r1=[]
p=[]
for j in range(len(poi_vals1)):
    t1=0
    for _ in range(poi_vals1[j]):
        t1+=weibull_generator(k,scale,vals[t])
        t+=1
    p.append(t1<5)
    r1.append(t1)

In [10]:
r2=[]
q=[]
for j in range(len(poi_vals2)):
    t1=0
    for _ in range(poi_vals2[j]):
        t1+=weibull_generator(k,scale,vals[t])
        t+=1
    q.append(t1<5)
    r2.append(t1)

In [11]:
print("probability for n=100",sum(p)/100)
print("probability for n=10000",sum(q)/10000)

probability for n=100 0.46
probability for n=10000 0.3671


In [12]:
mu1=np.mean(np.array(p))
mu2=np.mean(np.array(q))
s1=(np.sum(np.square(np.array(p)-mu1))/(len(poi_vals1)*(len(poi_vals1)-1)))**0.5
s2=(np.sum(np.square(np.array(q)-mu2))/(len(poi_vals2)*(len(poi_vals2)-1)))**0.5
print("Confidence Interval for n=100 is: (",mu1-2.58*s1,",",mu1+2.58*s1,")")
print("Confidence Interval for n=10000 is: (",mu2-2.58*s2,",",mu2+2.58*s2,")")

Confidence Interval for n=100 is: ( 0.33076566738179547 , 0.5892343326182046 )
Confidence Interval for n=10000 is: ( 0.3546634144447269 , 0.3795365855552731 )


In [13]:
from math import factorial as fac
def f(x):
    return np.exp(-2.9)*2.9**x/fac(x)
# def g(x,k,scale):
#     return k*scale*x**(k-1)*np.exp(-scale*x**k)

In [14]:
def generate_for_s(k1,t,s):
    q=[]
    j=0
    if s>5:
        while len(q)<k1:
            while poi_vals2[j]<6:
                j+=1
            t1=0
            for _ in range(poi_vals2[j]):
                t1+=weibull_generator(k,scale,vals[t])
                t+=1
            q.append(t1<5)
            j+=1
    else:
        # print(s)
        while len(q)<k1:
            t1=0
            for _ in range(s):
                t1+=weibull_generator(k,scale,vals[t])
                t+=1
            # print("here is t1",t1)
            q.append(t1<5)
    # print(k,scale)
    mu=np.mean(np.array(q))
    s=(np.sum(np.square(np.array(q)-mu))/(k1*(k1-1)))
    return [mu,s]
    

In [15]:
w7=int(100*(1-f(5)-f(4)-f(3)-f(2)-f(1)-f(0)))
p=1*f(0)
variance1=0
for s in range(1,7):
    p1=p
    if s<6:
        mu,var=generate_for_s(int(100*f(s)),20_000+s*100,s)
        p+=mu*f(s)
        variance1+=f(s)**2*var
    else:
        mu,var=generate_for_s(w7,20_000+700,7)
        p+=(1-f(5)-f(4)-f(3)-f(2)-f(1)-f(0))*mu
        variance1+=(1-f(5)-f(4)-f(3)-f(2)-f(1)-f(0))**2*var
    
        

In [16]:
w7=int(10000*(1-f(5)-f(4)-f(3)-f(2)-f(1)-f(0)))
q=1*f(0)
variance2=0
for s in range(1,7):
    p1=p
    if s<6:
        mu,var=generate_for_s(int(10000*f(s)),30_000+s*100,s)
        q+=mu*f(s)
        variance2+=f(s)**2*var
    else:
        mu,var=generate_for_s(w7,30_000+700,7)
        q+=(1-f(5)-f(4)-f(3)-f(2)-f(1)-f(0))*mu
        variance2+=(1-f(5)-f(4)-f(3)-f(2)-f(1)-f(0))**2*var
    
        

In [17]:
print("probability for n=100",p)
print("probability for n=10000",q)

probability for n=100 0.3842241300930192
probability for n=10000 0.37313533890378864


In [18]:
print("Confidence Interval for n=100 is: (",p-2.58*variance1,",",p+2.58*variance1,")")
print("Confidence Interval for n=10000 is: (",q-2.58*variance2,",",q+2.58*variance2,")")

Confidence Interval for n=100 is: ( 0.38069890113535837 , 0.38774935905068003 )
Confidence Interval for n=10000 is: ( 0.3730969265388537 , 0.3731737512687236 )


### Question2

In [19]:
alpha= [2082, 1999, 2008, 2047, 2199, 2153, 1999, 2136, 2053, 2121, 1974, 2110, 2110, 2168, 2035, 2019, 2044, 2191, 2284, 1912, 2196, 2099, 2041, 2192, 2188, 1984, 2158, 2019, 2032, 2051, 2192, 2133, 2142, 2113, 2150, 2221, 2046, 2127]

In [20]:
np.random.seed(73)
def gamma_gen(alpha,size):
    w=[]
    for _ in range(size):
        w.append(np.sum(-np.log(np.random.uniform(0,1,alpha))))
    return w
        
    
    
    

In [21]:
def find(arr,x):
    i=0
    while i<len(arr) and arr[i]<x:
        i+=1
    return i/len(arr)

In [22]:
y19=gamma_gen(alpha[18],5_000)
y=[]
for z in range(38):
    if z!=18:
        zz=gamma_gen(alpha[z],5_000)
        zz.sort()
        y.append(zz)


In [23]:
ans=[]
for y_val in y19:
    h=1
    for arr in y:
        h*=find(arr,y_val)
    ans.append(h)
    

In [24]:
mu=np.mean(ans)
var=np.var(ans)/len(ans)
print("mu is=",np.mean(ans))
print("variance is=",var)


mu is= 0.6341307657595057
variance is= 2.365457327277835e-05


### Question 3

In [25]:
log_norm=[]
mu=[2,0,0,1,2]
sigma2=[3,1,2,2,1]

for z in range(5):
    r=(-2*np.log(np.random.uniform(0,1,100_000)+1e-9))**0.5 #Adding a small value so that inside log it doesnt becomes 0
    theta=2*np.pi*np.random.uniform(0,1,100_000)
    box_muller=mu[z]+(sigma2[z]**0.5)*(r*np.cos(theta))
    log_norm.append(np.exp(box_muller))

In [26]:
fx=log_norm[0][:]
hx=log_norm[0][:]
for z in range(1,5):
    fx=fx+log_norm[z][:]
    hx=hx*log_norm[z][:]
fx=fx/5
hx=hx/5

In [27]:
fx

array([18.79072551,  5.90070181,  2.01711126, ..., 22.25438929,
       14.51641681, 24.66789679])

In [28]:
hx

array([  30.19071266,   25.98412701,    2.69290768, ..., 1691.70551829,
         30.55650575,   20.78160467])

In [29]:
mu_=np.mean(fx)

In [30]:
theta_=np.mean(hx)

In [31]:
beta_=np.sum((fx-mu_)*(hx-theta_))/(np.sum(np.square((hx-theta_))))

In [32]:
theta=np.exp(sum(mu) + 0.5 * sum(sigma2)) / 5

In [33]:
mean_= mu_+ beta_* (theta - theta_)

In [34]:
variance=np.sum(np.square(fx - mu_ - beta_ * (hx - theta_))) / (len(log_norm[0])**2)
l, r = mean_ - 2.58*np.sqrt(variance), mean_ + 2.58*np.sqrt(variance)

In [35]:
print("mu is=",mean_)
print("variance is=",variance)
print("with 99% confidence interval=(",l,",",r,")")

mu is= 11.444181125411916
variance is= 0.006912495807679744
with 99% confidence interval=( 11.22967626044555 , 11.658685990378283 )
